In [2]:
# 163

## 셀레늄을 이용한 자동화 크롤러_1

In [1]:
!pip install selenium

In [3]:
from selenium.webdriver import Chrome  # 웹 드라이버 로드 및 브라우저 기동(크롬 웹드라이버가 같은 디렉토리에 설치되어 있어야 함)
import time
import sqlite3
from pandas.io import sql
import os
import pandas as pd

In [4]:
from selenium import webdriver # 웹 드라이버 임포트

options = webdriver.ChromeOptions()  
options.add_argument("--start-maximized");  # 크롬 브라우저의 웹 창 크기를 최대화 -> 창이 작으면 반응형 웹에 의해 UI들이 바뀌기 때문

browser = webdriver.Chrome('chromedriver', options=options)  # 크롬 브라우저를 기동

In [5]:
browser.get('https://www.data.go.kr/')
browser.implicitly_wait(5)  # 묵시적으로 5초간 기다려라(타임슬립과 유사한 기능) -> 사이트에 따라 기동 속도가 다르기 때문에 잠깐 대기하는 용도

In [6]:
browser.find_element_by_xpath('//*[@id="header"]/div/div/div/div[2]/div/a[1]').click()  # 로그인 버튼의 xpath를 찾아서 클릭
browser.implicitly_wait(5)

In [7]:
browser.find_element_by_xpath('//*[@id="mberId"]').send_keys('아이디 입력')  # 아이디 입력

In [8]:
browser.find_element_by_xpath('//*[@id="pswrd"]').send_keys('비밀번호 입력')  # 비밀번호 입력

In [9]:
browser.find_element_by_xpath('//*[@id="loginVo"]/div[2]/div[2]/div[2]/div/div[1]/button').click()  # 로그인 버튼 클릭
browser.implicitly_wait(5)

In [10]:
browser.find_element_by_xpath('//*[@id="M000400_pc"]/a').click()  # 정보공유 클릭

In [11]:
browser.find_element_by_xpath('//*[@id="M000402_pc"]/a').click()  # 자료실 클릭

In [12]:
def db_save(ARTICLE_LIST):
    with sqlite3.connect(os.path.join('.','sqliteDB')) as con: # sqlite DB 파일이 존재하지 않는 경우 파일생성
        try:
            ARTICLE_LIST.to_sql(name = 'ARTICLE_LIST', con = con, index = False, if_exists='append') 
            #if_exists : {'fail', 'replace', 'append'} default : fail
        except Exception as e:
            print(str(e))
        print(len(ARTICLE_LIST), '건 저장완료..')

In [13]:
trs = browser.find_elements_by_xpath('//*[@id="searchVO"]/div[5]/table/tbody/tr') # 목록의 제목 xpath
df_list = []  # 빈 리스트 하나 생성
for tr in trs:
    df = pd.DataFrame({    # 판다스 데이터 프레임으로 만듦, 딕셔너리{} 타입으로.
            'NO': [tr.find_element_by_xpath('td[1]').text],   # 딕셔너리의 키:값 에서 '값'은 []로 감싸서 리스트 형태로 만들어줘야 함(중요!)
            'TITLE': [tr.find_element_by_xpath('td[2]').text],  # 제목
            'IQRY': [tr.find_element_by_xpath('td[3]').text],   # 조회수
            'REGDT': [tr.find_element_by_xpath('td[4]').text],  # 등록일
            'CHGDT': [tr.find_element_by_xpath('td[5]').text],  # 변경일
        })
    df_list.append(df)  # 위의 데이터 프레임을 빈 리스트에 저장
    
ARTICLE_LIST = pd.concat(df_list)  # pd.concat으로 여러 데이터 프레임이 담긴 리스트를 하나로 합침
db_save(ARTICLE_LIST)  # sqlite DB에 저장

10 건 저장완료..


In [14]:
browser.find_element_by_xpath('//*[@id="searchVO"]/div[5]/table/tbody/tr[1]/td[2]/a').click() # 글 클릭
browser.implicitly_wait(3)

In [16]:
browser.find_element_by_xpath('//*[@id="recsroomDetail"]/div[2]/div[4]/div/a').click()  # 해당 글의 첨부파일 클릭(다운로드)
time.sleep(10)

In [15]:
browser.quit()  # 종료. 그냥 창 닫는게 아니라 이 방법으로 종료해야 함.